In [8]:
!gpustat

seven                Wed Mar 10 21:04:10 2021  440.64
[0] TITAN RTX        | 36'C,   0 % |    11 / 24220 MB |
[1] TITAN RTX        | 84'C,  39 % |  2662 / 24220 MB | cwb(2651M)
[2] TITAN RTX        | 33'C,   0 % |    11 / 24220 MB |
[3] TITAN RTX        | 87'C,  99 % |  9928 / 24220 MB | cwb(3411M) cwb(3063M) cwb(3443M)


In [2]:
from os.path import join as pathjoin
from data_processing import *
from interpretation import *
from models import *
from training import *

In [3]:
ddd = pd.read_csv('/home/mlepekhin/data/ru_train')
ddd = ddd.append(pd.read_csv('/home/mlepekhin/data/en_train'))
ddd = ddd.append(pd.read_csv('/home/mlepekhin/data/new_ru_attacked_30.csv')).sample(frac=1, random_state=42)
print(ddd.shape)
ddd.head()

(3429, 7)


,Unnamed: 0,target,text,changed_words_num,new_model_target,old_model_target,old_text
578,804,A1,ОКОНЧАТЕЛЬНЫЙ ОТЧЕТ ОБ ОДНА ТЫСЯЧА СТО ПЯТЬДЕС...,NaN,NaN,NaN,NaN
1055,1916,A17,Отдыхали с мужем в конце октября / начале нояб...,NaN,NaN,NaN,NaN
564,1784,A14,"Исследуется движение материальной точки , подв...",NaN,NaN,NaN,NaN
851,1038,A9,WHEREAS by the Wimbledon and Putney Commons Ac...,NaN,NaN,NaN,NaN
532,532,A14,О ГЕОМЕТРИЧЕСКОЙ ФУНКЦИОНИРОВАНИЯ РЕГИСТРИРУЕ...,10.0,A12,A14,О ФАЗОВОЙ СТРУКТУРЕ РЕГИСТРИРУЕМОГО СПЕКТРА Л...


In [4]:
ddd.to_csv('ru_attacked_train_30.csv')

Let's download all the data.

In [5]:
DATA_DIR = '/home/mlepekhin/data'
MODELS_DIR = '/home/mlepekhin/models'
MODEL_ID = 'allennlp_xlm_roberta_ru_attacked_30' 
CHECKPOINTS_DIR = pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints')
BEST_MODEL = pathjoin(CHECKPOINTS_DIR, 'best.th')

In [6]:
transformer_model = 'xlm-roberta-base'
MAX_TOKENS = 512

In [7]:
tokenizer = PretrainedTransformerTokenizer(transformer_model)

03102021 21:04:31|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /home/mlepekhin/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.2b0f807393c56e8861a31cd67d2fc0b45d71d9735dd47dd66afb650f90b6d2a8
03102021 21:04:31|INFO|transformers.configuration_utils| Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 250002
}

03102021 21:04:32|INFO|transformers.tokenization_

In [8]:
train_dataset_reader = build_transformer_dataset_reader(transformer_model, lower=True)
val_dataset_reader = build_transformer_dataset_reader(transformer_model, lower=True)

train_data, dev_data = read_data(
    "ru_attacked_train_30.csv", 
    pathjoin(DATA_DIR, "manual_genres", "all.csv"),
    train_dataset_reader, 
    val_dataset_reader
)

vocab = build_vocab(train_data + dev_data)

train_data.index_with(vocab)
dev_data.index_with(vocab)

03102021 21:04:35|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /home/mlepekhin/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.2b0f807393c56e8861a31cd67d2fc0b45d71d9735dd47dd66afb650f90b6d2a8
03102021 21:04:35|INFO|transformers.configuration_utils| Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 250002
}

03102021 21:04:36|INFO|transformers.tokenization_

03102021 21:04:45|INFO|transformers.tokenization_utils| loading file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-sentencepiece.bpe.model from cache at /home/mlepekhin/.cache/torch/transformers/0c370616ddfc06067c0634160f749c2cf9d8da2c50e03a2617ce5841c8df3b1d.309f0c29486cffc28e1e40a2ab0ac8f500c203fe080b95f820aa9cb58e5b84ed
03102021 21:04:46|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /home/mlepekhin/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.2b0f807393c56e8861a31cd67d2fc0b45d71d9735dd47dd66afb650f90b6d2a8
03102021 21:04:46|INFO|transformers.configuration_utils| Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
 

Reading data
<class 'data_processing.ClassificationDatasetReader'> ru_attacked_train_30.csv


03102021 21:10:11|INFO|allennlp.data.vocabulary| Fitting token dictionary from dataset.



Building the vocabulary


In [9]:
model = build_pool_transformer_model(vocab, transformer_model)

Building the model


03102021 21:10:12|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/xlm-roberta-base-config.json from cache at /home/mlepekhin/.cache/torch/transformers/762ddd751172e9d3229e5da17a459eee6c0dfdc237c718944d0b1a85f06c7e1e.2b0f807393c56e8861a31cd67d2fc0b45d71d9735dd47dd66afb650f90b6d2a8
03102021 21:10:12|INFO|transformers.configuration_utils| Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 250002
}

03102021 21:10:12|INFO|transformers.modeling_util

In [10]:
if torch.cuda.is_available():
    cuda_device = 2
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
print(cuda_device)

2


In [11]:
!rm -rf {CHECKPOINTS_DIR}
!mkdir -p {CHECKPOINTS_DIR}

In [12]:
train_loader, dev_loader = build_data_loaders(train_data, dev_data)

# You obviously won't want to create a temporary file for your training
# results, but for execution in binder for this course, we need to do this.

trainer = build_classifier_trainer(
    model,
    pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints'),
    train_loader,
    dev_loader,
    10,
    cuda_device=cuda_device
)
print("Starting training")
trainer.train()
print("Finished training")

03102021 21:10:30|INFO|allennlp.training.optimizers| Number of trainable parameters: 278052107
03102021 21:10:30|WARNING|allennlp.training.trainer| You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
03102021 21:10:30|INFO|allennlp.training.trainer| Beginning training.
03102021 21:10:30|INFO|allennlp.training.trainer| Epoch 0/9
03102021 21:10:30|INFO|allennlp.training.trainer| Worker 0 memory usage MB: 5849.7


Starting training


03102021 21:10:30|INFO|allennlp.training.trainer| GPU 0 memory usage MB: 11
03102021 21:10:30|INFO|allennlp.training.trainer| GPU 1 memory usage MB: 2662
03102021 21:10:30|INFO|allennlp.training.trainer| GPU 2 memory usage MB: 1990
03102021 21:10:30|INFO|allennlp.training.trainer| GPU 3 memory usage MB: 9928
03102021 21:10:30|INFO|allennlp.training.trainer| Training


03102021 21:12:28|INFO|allennlp.training.trainer| Validating


03102021 21:13:08|INFO|allennlp.training.tensorboard_writer|                        Training |  Validation
03102021 21:13:08|INFO|allennlp.training.tensorboard_writer| accuracy           |     0.684  |     0.732
03102021 21:13:08|INFO|allennlp.training.tensorboard_writer| gpu_0_memory_MB    |    11.000  |       N/A
03102021 21:13:08|INFO|allennlp.training.tensorboard_writer| gpu_1_memory_MB    |  2662.000  |       N/A
03102021 21:13:08|INFO|allennlp.training.tensorboard_writer| gpu_2_memory_MB    |  1990.000  |       N/A
03102021 21:13:08|INFO|allennlp.training.tensorboard_writer| gpu_3_memory_MB    |  9928.000  |       N/A
03102021 21:13:08|INFO|allennlp.training.tensorboard_writer| loss               |     1.011  |     0.818
03102021 21:13:08|INFO|allennlp.training.tensorboard_writer| reg_loss           |     0.000  |     0.000
03102021 21:13:08|INFO|allennlp.training.tensorboard_writer| worker_0_memory_MB |  5849.700  |       N/A


...

03102021 21:35:13|INFO|allennlp.training.trainer| Epoch duration: 0:02:43.871509
03102021 21:35:13|INFO|allennlp.training.trainer| Estimated training time remaining: 0:02:44
03102021 21:35:13|INFO|allennlp.training.trainer| Epoch 9/9
03102021 21:35:13|INFO|allennlp.training.trainer| Worker 0 memory usage MB: 5961.608
03102021 21:35:13|INFO|allennlp.training.trainer| GPU 0 memory usage MB: 18526
03102021 21:35:13|INFO|allennlp.training.trainer| GPU 1 memory usage MB: 2662
03102021 21:35:13|INFO|allennlp.training.trainer| GPU 2 memory usage MB: 18526
03102021 21:35:13|INFO|allennlp.training.trainer| GPU 3 memory usage MB: 9928
03102021 21:35:13|INFO|allennlp.training.trainer| Training


03102021 21:37:13|INFO|allennlp.training.trainer| Validating


03102021 21:37:53|INFO|allennlp.training.tensorboard_writer|                        Training |  Validation
03102021 21:37:53|INFO|allennlp.training.tensorboard_writer| accuracy           |     0.970  |     0.743
03102021 21:37:53|INFO|allennlp.training.tensorboard_writer| gpu_0_memory_MB    |  18526.000  |       N/A
03102021 21:37:53|INFO|allennlp.training.tensorboard_writer| gpu_1_memory_MB    |  2662.000  |       N/A
03102021 21:37:53|INFO|allennlp.training.tensorboard_writer| gpu_2_memory_MB    |  18526.000  |       N/A
03102021 21:37:53|INFO|allennlp.training.tensorboard_writer| gpu_3_memory_MB    |  9928.000  |       N/A
03102021 21:37:53|INFO|allennlp.training.tensorboard_writer| loss               |     0.090  |     1.206
03102021 21:37:53|INFO|allennlp.training.tensorboard_writer| reg_loss           |     0.000  |     0.000
03102021 21:37:53|INFO|allennlp.training.tensorboard_writer| worker_0_memory_MB |  5961.608  |       N/A


03102021 21:37:57|INFO|allennlp.training.trainer| Epoch duration: 0:02:43.738384
03102021 21:37:57|INFO|allennlp.training.checkpointer| loading best weights


Finished training


In [13]:
!rm "{CHECKPOINTS_DIR}"/*.json "{CHECKPOINTS_DIR}"/model* "{CHECKPOINTS_DIR}"/training*
model.vocab.save_to_files(pathjoin(MODELS_DIR, MODEL_ID, 'vocab'))
!ls -lh "{CHECKPOINTS_DIR}"

03102021 21:38:00|INFO|filelock| Lock 139815299969472 acquired on /home/mlepekhin/models/allennlp_xlm_roberta_ru_attacked_30/vocab/.lock
03102021 21:38:00|INFO|filelock| Lock 139815299969472 released on /home/mlepekhin/models/allennlp_xlm_roberta_ru_attacked_30/vocab/.lock


total 1,1G
-rw-rw-r-- 1 mlepekhin mlepekhin 1,1G мар 10 21:15 best.th
drwxrwxr-x 4 mlepekhin mlepekhin 4,0K мар 10 21:10 log
